In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

from ecmwfapi import ECMWFDataServer
import os
import sys
import dask
import numpy as np
from esio import download
import datetime
from esio import EsioData as ed
import pandas as pd

import calendar
import datetime

import os
from ecmwfapi import ECMWFDataServer

'''
Download models with sea ice forecasts within the 2s2 forecast data archive.

https://software.ecmwf.int/wiki/display/S2S/Models

'''

'\nDownload models with sea ice forecasts within the 2s2 forecast data archive.\n\nhttps://software.ecmwf.int/wiki/display/S2S/Models\n\n'

In [2]:
# meteofr always uses same version which is 2014-12-01
# from 1993 to one year ago

# the MVDs are on 1, 8, 15, 22 
dataclass='s2s'
run_type='reforecast'
cmod = 'metreofr'

versions = datetime.datetime(2014, 12, 1) # 
print(versions)
vdate = versions.strftime('%Y-%m-%d')
print(vdate)

2014-12-01 00:00:00
2014-12-01


In [3]:
E = ed.EsioData.load()
main_dir = E.model_dir

In [4]:
# a starting point
mod_dicts = {}

# UKMO
mod_dicts['metreofr'] = {
 "class": "s2",
 "dataset": "s2s",
 "date": "2014-12-01",                                                               
 "hdate": "2005-07-11",                                                          
 "expver": "prod",
 "levtype": "sfc",
 "model": "glob",
 "origin": "lfpw",
 "param": "31",
 "step": "0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-264/264-288/288-312/312-336/336-360/360-384/384-408/408-432/432-456/456-480/480-504/504-528/528-552/552-576/576-600/600-624/624-648/648-672/672-696/696-720/720-744/744-768/768-792/792-816/816-840/840-864/864-888/888-912/912-936/936-960/960-984/984-1008/1008-1032/1032-1056/1056-1080/1080-1104/1104-1128/1128-1152/1152-1176/1176-1200/1200-1224/1224-1248/1248-1272/1272-1296/1296-1320/1320-1344/1344-1368/1368-1392/1392-1416/1416-1440",
 "stream": "enfh",
 "number": "1/2/3/4/5/6/7/8/9/10/11/12/13/14",
 "time": "00:00:00",
 "type": "pf",
 "target": "output",
 }


In [5]:
@dask.delayed

def download_month(config_dict):
    # Start server
    cserver = ECMWFDataServer()
    cserver.retrieve(config_dict)
    return 1

In [12]:
cy=1997
DS = datetime.datetime(cy, 1, 1) # 
DE = datetime.datetime(cy, 12, 28) 

earlyinmonth = pd.date_range(start=DS,end=DE,freq='SMS-8')  # 1st and 8th of month
laterinmonth = pd.date_range(start=DS,end=DE,freq='SMS-15')  # 1st and 15th of month
evenlaterinmonth = pd.date_range(start=DS,end=DE,freq='SMS-22')  # 1st and 22th of month
alldates=np.append(earlyinmonth,laterinmonth)               #smoosh together
alldates=np.unique(np.append(alldates,evenlaterinmonth))    #sort and ditch duplicates
alldates = pd.DatetimeIndex(alldates)
hdates = [x.strftime('%Y-%m-%d') for x in alldates]
#hdates=hdates[0:24]
hdates=hdates[24:48]

print(hdates)
len(hdates)


['1997-06-22', '1997-07-01', '1997-07-08', '1997-07-15', '1997-07-22', '1997-08-01', '1997-08-08', '1997-08-15', '1997-08-22', '1997-09-01', '1997-09-08', '1997-09-15', '1997-09-22', '1997-10-01', '1997-10-08', '1997-10-15', '1997-10-22', '1997-11-01', '1997-11-08', '1997-11-15', '1997-11-22', '1997-12-01', '1997-12-08', '1997-12-15', '1997-12-22']


25

['1997-01-01', '1997-01-08', '1997-01-15', '1997-01-22', '1997-02-01', '1997-02-08', '1997-02-15', '1997-02-22', '1997-03-01', '1997-03-08', '1997-03-15', '1997-03-22', '1997-04-01', '1997-04-08', '1997-04-15', '1997-04-22', '1997-05-01', '1997-05-08', '1997-05-15', '1997-05-22', '1997-06-01', '1997-06-08', '1997-06-15', '1997-06-22', '1997-07-01', '1997-07-08', '1997-07-15', '1997-07-22', '1997-08-01', '1997-08-08', '1997-08-15', '1997-08-22', '1997-09-01', '1997-09-08', '1997-09-15', '1997-09-22', '1997-10-01', '1997-10-08', '1997-10-15', '1997-10-22', '1997-11-01', '1997-11-08', '1997-11-15', '1997-11-22', '1997-12-01', '1997-12-08', '1997-12-15', '1997-12-22']


In [16]:

for cy in np.arange(1996,1997,1):

    DS = datetime.datetime(cy, 1, 1) # 
    DE = datetime.datetime(cy, 12, 28) 

    earlyinmonth = pd.date_range(start=DS,end=DE,freq='SMS-8')  # 1st and 8th of month
    laterinmonth = pd.date_range(start=DS,end=DE,freq='SMS-15')  # 1st and 15th of month
    evenlaterinmonth = pd.date_range(start=DS,end=DE,freq='SMS-22')  # 1st and 22th of month
    alldates=np.append(earlyinmonth,laterinmonth)               #smoosh together
    alldates=np.unique(np.append(alldates,evenlaterinmonth))    #sort and ditch duplicates
    alldates = pd.DatetimeIndex(alldates)
    hdates = [x.strftime('%Y-%m-%d') for x in alldates]
    #hdates=hdates[0:23]  # cut in half
    hdates=hdates[24:48]  # cut in half
    print(hdates)

    X = 1

    for hd in np.arange(0,len(hdates),1):

        cdict = mod_dicts[cmod]
        cdict['date'] = vdate
        cdict['hdate'] = hdates[hd]

        target = os.path.join(main_dir, cmod, run_type, 'native',
                                  cmod+'_'+hdates[hd]+'.grib')
        cdict['target'] = target
        cdict['expect'] = 'any'

        print('Version Date is ',vdate,' hdate is ',hdates[hd])
        print('output file is ',target)
        print('/n/n')
        #X = download_month(cdict)
        #X.compute()
        X = X + download_month(cdict)

    # Call compute to download all years concurently for this date
    X.compute()

['1996-07-01', '1996-07-08', '1996-07-15', '1996-07-22', '1996-08-01', '1996-08-08', '1996-08-15', '1996-08-22', '1996-09-01', '1996-09-08', '1996-09-15', '1996-09-22', '1996-10-01', '1996-10-08', '1996-10-15', '1996-10-22', '1996-11-01', '1996-11-08', '1996-11-15', '1996-11-22', '1996-12-01', '1996-12-08', '1996-12-15', '1996-12-22']
Version Date is  2014-12-01  hdate is  1996-07-01
output file is  /home/disk/sipn/nicway/data/model/metreofr/reforecast/native/metreofr_1996-07-01.grib
/n/n
Version Date is  2014-12-01  hdate is  1996-07-08
output file is  /home/disk/sipn/nicway/data/model/metreofr/reforecast/native/metreofr_1996-07-08.grib
/n/n
Version Date is  2014-12-01  hdate is  1996-07-15
output file is  /home/disk/sipn/nicway/data/model/metreofr/reforecast/native/metreofr_1996-07-15.grib
/n/n
Version Date is  2014-12-01  hdate is  1996-07-22
output file is  /home/disk/sipn/nicway/data/model/metreofr/reforecast/native/metreofr_1996-07-22.grib
/n/n
Version Date is  2014-12-01  hdate 

Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-20 15:27:49 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
2019-07-20 15:27:49 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https://apps.ecmwf.int/webapi-activity/
2019-07-20 15:27:49 In case of problems, please check https://confluence.ecmwf.int/display/WEBAPI/Web-API+Troubleshooting or contact servicedesk@ecmwf.int. In case of delays, please check  service activity page https:/

Calling 'nice mars /tmp/3f/1d/tmp-_mars2JEdPB.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190720.222911 - Welcome to MARS
mars - INFO   - 20190720.222911 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190720.222911 - MARS Client version: 6.25.3
mars - INFO   - 20190720.222911 - MIR version: 1.2.5
mars - INFO   - 20190720.222911 - Using ecCodes version 2.12.6
mars - INFO   - 20190720.222911 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190720.222911 - Maximum retrieval size is 30.00 G
retrieve,origin=lfpw,hdate=1996-08-01,stream=enfh,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/11/12/13/14,expver=prod,padding=0,step=0-24/24-48/48-72/72-96/96-120/120-144/144-168/168-192/192-216/216-240/240-26

2019-07-20 15:29:53 Request is queued
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-20 15:30:09 Transfer rate 3.00879 Mbytes/s
2019-07-20 15:30:09 Transfer rate 3.0207 Mbytes/s
2019-07-20 15:30:09 Transfer rate 2.9914 Mbytes/s
2019-07-20 15:30:09 Transfer rate 3.0159 Mbytes/s
2019-07-20 15:30:09 Transfer rate 3.038 Mbytes/s
2019-07-20 15:30:10 Transfer rate 3.01538 Mbytes/s
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 s

2019-07-20 15:30:56 Request is queued
Calling 'nice mars /tmp/c7/bf/tmp-_marsWeabJZ.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190720.222953 - Welcome to MARS
mars - INFO   - 20190720.222953 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190720.222953 - MARS Client version: 6.25.3
mars - INFO   - 20190720.222953 - MIR version: 1.2.5
mars - INFO   - 20190720.222953 - Using ecCodes version 2.12.6
mars - INFO   - 20190720.222953 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190720.222953 - Maximum retrieval size is 30.00 G
retrieve,origin=lfpw,hdate=1996-09-08,stream=enfh,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/11/12/13/14,expver=prod,padding=0,step=0-24/24-48/48-72/72-96/96-120/120-144/

...

2019-07-20 15:32:13 Request submitted
2019-07-20 15:32:13 Request id: 5d33966dc7db1c98d3507ddc
2019-07-20 15:32:13 Request is submitted
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
2019-07-20 15:32:14 ECMWF API python library 1.5.4
2019-07-20 15:32:14 ECMWF API at https://api.ecmwf.int/v1
2019-07-20 15:32:14 Request submitted
2019-07-20 15:32:14 Request id: 5d33966ebbd91cde3b0864cc
2019-07-20 15:32:14 Request is submitted
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Error contacting the WebAPI, retrying in 60 seconds ...
Err

2019-07-20 15:33:16 Request is queued
2019-07-20 15:33:16 Request is queued
2019-07-20 15:33:16 Request is queued
Calling 'nice mars /tmp/18/89/tmp-_marsJprnDN.req'
mars - WARN -
mars - WARN - From 29 January 2019 10AM (UTC) MARS uses the interpolation
mars - WARN - provided by the MIR library. For more details, see
mars - WARN - https://confluence.ecmwf.int/display/UDOC/MARS+interpolation+with+MIR
mars - WARN -
PPDIR is /var/tmp/ppdir/x86_64
mars - INFO   - 20190720.223217 - Welcome to MARS
mars - INFO   - 20190720.223217 - MARS Client build stamp: 20190610154938
mars - INFO   - 20190720.223217 - MARS Client version: 6.25.3
mars - INFO   - 20190720.223217 - MIR version: 1.2.5
mars - INFO   - 20190720.223217 - Using ecCodes version 2.12.6
mars - INFO   - 20190720.223217 - Using odb_api version: 0.15.10 (file format version: 0.5)
mars - INFO   - 20190720.223217 - Maximum retrieval size is 30.00 G
retrieve,origin=lfpw,hdate=1996-11-08,stream=enfh,levtype=sfc,number=1/2/3/4/5/6/7/8/9/10/1

mars - INFO   - 20190720.223325 - 840 fields retrieved from 'marsth'
mars - WARN   - 20190720.223325 - Visiting database marsth : expected 0, got 840
mars - INFO   - 20190720.223325 - Request time:  wall: 8 sec  cpu: 3 sec
mars - INFO   - 20190720.223325 -   Read from network: 49.66 Mbyte(s) in 5 sec  [10.56 Mbyte/sec]
mars - INFO   - 20190720.223325 -   Visiting marsth: wall: 8 sec
mars - INFO   - 20190720.223325 -   Post-processing: wall: 2 sec cpu: 2 sec
mars - INFO   - 20190720.223325 -   Writing to target file: 49.66 Mbyte(s) in < 1 sec [677.82 Mbyte/sec]
mars - INFO   - 20190720.223325 - Memory used: 35.51 Mbyte(s)
mars - INFO   - 20190720.223325 - No errors reported
Process '['nice', 'mars', '/tmp/24/72/tmp-_marsI3uZrX.req']' finished
2019-07-20 15:33:28 Request is complete
2019-07-20 15:33:28 Transfering 49.6561 Mbytes into /home/disk/sipn/nicway/data/model/metreofr/reforecast/native/metreofr_1996-11-15.grib
2019-07-20 15:33:28 From https://stream.ecmwf.int/data/atls05/data/dat

mars - INFO   - 20190720.223337 - 840 fields retrieved from 'marsth'
mars - WARN   - 20190720.223337 - Visiting database marsth : expected 0, got 840
mars - INFO   - 20190720.223337 - Request time:  wall: 8 sec  cpu: 3 sec
mars - INFO   - 20190720.223337 -   Read from network: 49.65 Mbyte(s) in 4 sec  [11.52 Mbyte/sec]
mars - INFO   - 20190720.223337 -   Visiting marsth: wall: 8 sec
mars - INFO   - 20190720.223337 -   Post-processing: wall: 2 sec cpu: 2 sec
mars - INFO   - 20190720.223337 -   Writing to target file: 49.65 Mbyte(s) in < 1 sec [558.81 Mbyte/sec]
mars - INFO   - 20190720.223337 - Memory used: 35.51 Mbyte(s)
mars - INFO   - 20190720.223337 - No errors reported
Process '['nice', 'mars', '/tmp/9b/c7/tmp-_marsgk0VbC.req']' finished
2019-07-20 15:33:41 Request is complete
2019-07-20 15:33:41 Transfering 49.6537 Mbytes into /home/disk/sipn/nicway/data/model/metreofr/reforecast/native/metreofr_1996-12-22.grib
2019-07-20 15:33:41 From https://stream.ecmwf.int/data/atls04/data/dat

KeyboardInterrupt: 